In [1]:
import neurokit2 as nk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Paths to folders
raw_data_folder = '/Users/firefly/Desktop/Team_Messung/data/rawdata'
results_folder = '/Users/firefly/Desktop/Team_Messung/data/results'
os.makedirs(results_folder, exist_ok=True)

# Parameters
participants = ['sub-01', 'sub-02']
conditions1 = ['empty', 'full']
conditions2 = ['bright', 'dark']

all_results = []

# Process each participant and condition
for pi in participants:
    for c1 in conditions1:
        for c2 in conditions2:
            # Assemble file name
            filename = f"{raw_data_folder}/{pi}_{c1}-{c2}_eda.csv"
            print(f'\nProcessing: {filename}')
            
            try:
                # Read EDA data
                subdata = pd.read_csv(filename, header=None, names=['EDA'], skiprows=1)
                eda_data = subdata['EDA'].values
                
                print(f"Data length: {len(eda_data)} samples")
                
                # Process the EDA signal
                signals, info = nk.eda_process(eda_data, sampling_rate=1000)
                
                # Create figure
                fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(15, 10))
                
                # Plot raw and cleaned signal
                ax0.plot(signals["EDA_Raw"], label="Raw", color='lightgray')
                ax0.plot(signals["EDA_Clean"], label="Cleaned", color='dodgerblue')
                ax0.set_title("Raw and Cleaned EDA Signal")
                ax0.set_ylabel("Amplitude (µS)")
                ax0.legend()
                ax0.grid(True)
                
                # Plot Phasic and Tonic components
                ax1.plot(signals["EDA_Tonic"], label="Tonic", color='red', linewidth=2)
                ax1.plot(signals["EDA_Phasic"], label="Phasic", color='green')
                
                # Plot SCR peaks if available
                if "SCR_Peaks" in signals.columns:
                    peaks = np.where(signals["SCR_Peaks"] == 1)[0]
                    ax1.scatter(peaks, signals["EDA_Phasic"][peaks], 
                              c='orange', label="SCR Peaks", zorder=3)
                
                ax1.set_title("Tonic and Phasic Components")
                ax1.set_xlabel("Samples")
                ax1.set_ylabel("Amplitude (µS)")
                ax1.legend()
                ax1.grid(True)
                
                # Add overall title
                plt.suptitle(f"EDA Analysis - {pi} ({c1}_{c2})", y=0.95)
                plt.tight_layout()
                
                # Save figure
                figure_filename = f"{results_folder}/{pi}_{c1}_{c2}_eda_nk.png"
                plt.savefig(figure_filename, dpi=300, bbox_inches='tight')
                print(f"Saved figure to {figure_filename}")
                plt.close()
                
                # Process EDA features
                print("\nCalculating EDA features...")
                results = nk.eda_intervalrelated(signals, sampling_rate=1000)
                
                # Add metadata
                results['Participant'] = pi
                results['Condition'] = f"{c1}_{c2}"
                all_results.append(results)
                
                # Print some key metrics
                print("\nKey metrics:")
                print(f"Number of SCRs: {len(peaks) if 'SCR_Peaks' in signals.columns else 0}")
                print(f"Mean EDA: {np.mean(signals['EDA_Clean']):.3f} µS")
                print(f"Mean Tonic: {np.mean(signals['EDA_Tonic']):.3f} µS")
                print(f"Mean Phasic: {np.mean(signals['EDA_Phasic']):.3f} µS")
                
            except FileNotFoundError:
                print(f'File not found: {filename}')
                continue
            except Exception as e:
                print(f'Error processing {filename}: {str(e)}')
                continue

# Combine all results
print("\nCombining all results...")
final_results = pd.concat(all_results, ignore_index=True)

# Save results
output_filename = f"{results_folder}/eda_results.csv"
final_results.to_csv(output_filename, index=False)
print(f"\nSaved results to {output_filename}")


Processing: /Users/firefly/Desktop/Team_Messung/data/rawdata/sub-01_empty-bright_eda.csv
Data length: 299000 samples
Saved figure to /Users/firefly/Desktop/Team_Messung/data/results/sub-01_empty_bright_eda_nk.png

Calculating EDA features...

Key metrics:
Number of SCRs: 51
Mean EDA: 144.636 µS
Mean Tonic: 144.636 µS
Mean Phasic: 0.002 µS

Processing: /Users/firefly/Desktop/Team_Messung/data/rawdata/sub-01_empty-dark_eda.csv
Data length: 239000 samples
Saved figure to /Users/firefly/Desktop/Team_Messung/data/results/sub-01_empty_dark_eda_nk.png

Calculating EDA features...

Key metrics:
Number of SCRs: 52
Mean EDA: 133.839 µS
Mean Tonic: 133.836 µS
Mean Phasic: 0.000 µS

Processing: /Users/firefly/Desktop/Team_Messung/data/rawdata/sub-01_full-bright_eda.csv
Data length: 359000 samples
Saved figure to /Users/firefly/Desktop/Team_Messung/data/results/sub-01_full_bright_eda_nk.png

Calculating EDA features...

Key metrics:
Number of SCRs: 7
Mean EDA: 132.745 µS
Mean Tonic: 132.668 µS
Mea